Check Login

In [1]:
!huggingface-cli whoami

locchuong


In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
# Check if a GPU is available
if torch.cuda.is_available():
    # Get the current device index (default is 0 if no other device is specified)
    current_device = torch.cuda.current_device()
    
    # Get the name of the GPU at this device index
    gpu_name = torch.cuda.get_device_name(current_device)
    print(f"Current GPU: {gpu_name}")
else:
    print("No GPU available.")

Current GPU: Tesla P40


Download dataset

In [3]:
# !huggingface-cli download dataset longquan/llm-japanese-dataset-split_10
from datasets import load_dataset

# Make sure to provide the correct `repo_id`
dataset = load_dataset("longquan/llm-japanese-dataset-split_10")

README.md:   0%|          | 0.00/116 [00:00<?, ?B/s]

(…)-00000-of-00001-4aff41abcbe8fcb8.parquet:   0%|          | 0.00/63.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251655 [00:00<?, ? examples/s]

In [7]:
# Load dataset from local cache
dataset = load_dataset("longquan/llm-japanese-dataset-split_10", cache_dir="~/.cache/huggingface/datasets")

# View available dataset splits
print("Available Splits:", dataset.keys())

# Load specific split (e.g., 'train') and inspect the first few rows
train_data = dataset["train"]
print("First Rows of Train Split:", train_data[0])


Available Splits: dict_keys(['train'])
First Rows of Train Split: {'instruction': '次の日本語には間違っている部分があります。その部分を直して、正しい日本語の文を出力してください。', 'input': 'フライデースペシャル』（ニッポン放送）内で、１４：４０頃から放送されていた生放送コーナー「森野熊八\u3000体いきいき楽しい食卓」が２０１４年１０月にて終了となり、後コーナーとし２０１４年１１月より開始された事前録音フロート番組。', 'output': 'フライデースペシャル』（ニッポン放送）内で、１４：４０頃から放送されていた生放送コーナー「森野熊八\u3000体いきいき楽しい食卓」が２０１４年１０月にて終了となり、後コーナーとして２０１４年１１月より開始された事前録音フロート番組。'}


Load model and inference

In [9]:
from transformers import pipeline

In [10]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

In [11]:
pipe = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
)

In [12]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Please summarize the following text: 'Artificial intelligence is a field of computer science that enables machines to perform tasks that typically require human intelligence.'"}
]

messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': "Please summarize the following text: 'Artificial intelligence is a field of computer science that enables machines to perform tasks that typically require human intelligence.'"}]

In [13]:
outputs = pipe(
    messages,
    max_length=128,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "Here's a summary of the text:\n\nArtificial intelligence (AI) is a branch of computer science that allows machines to think and learn like humans. It involves developing algorithms and systems that can perform tasks that typically require human intelligence, such as problem-solving, decision-making, and learning."}


Run a sample on dataset

In [22]:
sample = train_data[0]
sample['output']

'フライデースペシャル』（ニッポン放送）内で、１４：４０頃から放送されていた生放送コーナー「森野熊八\u3000体いきいき楽しい食卓」が２０１４年１０月にて終了となり、後コーナーとして２０１４年１１月より開始された事前録音フロート番組。'

In [21]:
messages = [
    {"role": "system", "content": sample["instruction"]},
    {"role": "user", "content": sample["input"]}
]
messages

[{'role': 'system',
  'content': '次の日本語には間違っている部分があります。その部分を直して、正しい日本語の文を出力してください。'},
 {'role': 'user',
  'content': 'フライデースペシャル』（ニッポン放送）内で、１４：４０頃から放送されていた生放送コーナー「森野熊八\u3000体いきいき楽しい食卓」が２０１４年１０月にて終了となり、後コーナーとし２０１４年１１月より開始された事前録音フロート番組。'}]

In [24]:
outputs = pipe(
    messages,
    max_new_tokens=128,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': '「森野熊八\u3000体いきいき楽しい食卓」は、ニッポン放送（NHK）の「フライデースペシャル」内で、１４：４０頃から放送されていた生放送コーナーで、２０１４年１０月に終了した。 \n\nその後、２０１４年１１月に、事前録音フロート番組として始まった。'}


Random pick a sample

In [25]:
import random

In [37]:
random_sample = train_data[random.choice(range(train_data.num_rows))]

print("Reference: ",sample['output'],"\n")

messages = [
    {"role": "system", "content": random_sample["instruction"]},
    {"role": "user", "content": random_sample["input"]}
]
outputs = pipe(
    messages,
    max_new_tokens=128,
)
print("Model: ", outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Reference:  フライデースペシャル』（ニッポン放送）内で、１４：４０頃から放送されていた生放送コーナー「森野熊八　体いきいき楽しい食卓」が２０１４年１０月にて終了となり、後コーナーとして２０１４年１１月より開始された事前録音フロート番組。 

Model:  {'role': 'assistant', 'content': '佐藤優里亜（さいとう うりあ）とは、日本の女性声優です。彼女の声優業は、多くのゲーム、テレビ番組、映画などで活躍しています。\n\n佐藤優里亜は、声優業の始めは、NHK（日本放送協会）でTV番組にゲストとして出演したことで始まりました。後に、ゲームのキャラクターとしての声優業に進みました。\n\n佐藤優里亜の主なキャラクターとしては、スピーチボール、ミスターハンサン、そしてヒ'}


References

https://huggingface.co/datasets/longquan/llm-japanese-dataset-split_10